In [1]:
import os 
import pandas as pd 
import numpy as np

## Functions

In [2]:
def create_description_table(table, waves, language):
    table = table.copy()
    # drop all observations that are not from the desired waves
    table = table.fillna("-")
    # drop columns
    table = table.drop(["label_english","categories_english"], axis=1)
    table = table[table["question_group"] != "-"]
    table = table[table["variable"] != "-"]
    for i in [1,2,3]:
        if i not in waves:
            table = table.drop([f"wave{i}"], axis =1)
            
    # drop observations
    if len(waves) == 1:
        table = table[table[f"wave{waves[0]}"] != "-"]
    elif len(waves) == 2:
        table =table[(table[f"wave{waves[0]}"] != "-") | (table[f"wave{waves[1]}"] != "-")]          
    else: 
        table =table[
            (table[f"wave{waves[0]}"]!= "-") | (table[f"wave{waves[1]}"]!= "-") | (table[f"wave{waves[2]}"]!= "-")]
                                                                                
    # replace question id with question group id
    for i in waves:
        table[f"wave{i}"] = np.where(table[f"wave{i}"] == "-", table[f"wave{i}"], table["question_group"])
                                                                                                                                                  
    # create question link ":ref:`wNL-{question_group}`
    for i in waves:
        table[f"wave{i}"] = table[f'wave{i}'].apply(lambda x: f":ref:`w{str(i)}{language}-{x}`")
        table[f"wave{i}"] = np.where(table[f"wave{i}"].str.contains("--")==False, table[f"wave{i}"], "-")
    table = table.drop(["question_group"], axis=1)
    return table

## Create Table 

Read in description table 

In [3]:
table =pd.read_csv("description-tables/xyx-corona-questionnaire_renaming_with_groups.csv", sep=";")

Rename columns so they are easier to handle for the functions

In [4]:
table = table.rename(columns={
                  "L_CoronavirusImpact_wave3_3p.dta":"wave3", 
                  "L_CoronavirusImpact_4.0p.dta":"wave2", 
                  "L_CoronavirusImpact_3p.dta":"wave1",
                  "new_name": "variable"
                 })
    

Select wave and language indicator. `language` should be one lower-case letter. `waves` should be a list of one or more values.

In [5]:
waves = [3]
language = "e"

Create decription table and save to csv.

In [6]:
variable_table = create_description_table(table, waves ,language)

In [7]:
if len(waves) == 1:
    path = f"../docs/source/wave-{waves[0]}/"
else:
    path = f"../docs/source/"

In [8]:
variable_table

,variable,reference period other than survey period,wave3,type,ordered
5,p_2m_infected,-,:ref:`w3e-q1header`,float,-
6,p_2m_acquaintance_infected,-,:ref:`w3e-q1header`,float,-
7,p_2m_hospital_if_infect_self,-,:ref:`w3e-q1header`,float,-
8,p_2m_infected_and_pass_on,-,:ref:`w3e-q1header`,float,-
34,approp_gov_medical,-,:ref:`w3e-q33`,Categorical,True
...,...,...,...,...,...
934,teacher_platform_sec_child5,-,:ref:`w3e-v6_bokind`,bool,-
935,teacher_pickup_sec_child5,-,:ref:`w3e-v6_bokind`,bool,-
936,teacher_none_sec_child5,-,:ref:`w3e-v6_bokind`,bool,-
937,teacher_other_sec_child5,-,:ref:`w3e-v6_bokind`,bool,-


In [9]:
variable_table.to_csv(
    f"{path}variable_table-waves-{'-'.join(str(i) for i in waves)}-{language}.csv", sep=",", index=False)